In [3]:
from __future__ import division #, print_function
import sys
import os
import csv
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from fiona.crs import from_epsg
import geopandas as gp
import shapely as shp
from geopandas.tools import sjoin
from shapely.geometry import Point

import pylab as pl
%pylab inline

#import seaborn
#from scipy.optimize import curve_fit, minimize

Populating the interactive namespace from numpy and matplotlib


/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['datetime', 'choice']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [14]:
#gt_1301 = pd.read_csv('data_raw_green/gt_201601.csv')
#gt_1302 = pd.read_csv('data_raw_green/gt_201602.csv')
#gt_1303 = pd.read_csv('data_raw_green/gt_201603.csv')
#gt_1304 = pd.read_csv('data_raw_green/gt_201604.csv')
#gt_1305 = pd.read_csv('data_raw_green/gt_201605.csv')
#gt_1306 = pd.read_csv('data_raw_green/gt_201606.csv')
#gt_1307 = pd.read_csv('data_raw_green/gt_201607.csv')
gt_1308 = pd.read_csv('data_raw_green/gt_201308.csv')
gt_1309 = pd.read_csv('data_raw_green/gt_201309.csv')
gt_1310 = pd.read_csv('data_raw_green/gt_201310.csv')
gt_1311 = pd.read_csv('data_raw_green/gt_201311.csv')
gt_1312 = pd.read_csv('data_raw_green/gt_201312.csv')

In [15]:
gt_1401 = pd.read_csv('data_raw_green/gt_201401.csv')
gt_1402 = pd.read_csv('data_raw_green/gt_201402.csv')
gt_1403 = pd.read_csv('data_raw_green/gt_201403.csv')
gt_1404 = pd.read_csv('data_raw_green/gt_201404.csv')
gt_1405 = pd.read_csv('data_raw_green/gt_201405.csv')
gt_1406 = pd.read_csv('data_raw_green/gt_201406.csv')
gt_1407 = pd.read_csv('data_raw_green/gt_201407.csv')
gt_1408 = pd.read_csv('data_raw_green/gt_201408.csv')
gt_1409 = pd.read_csv('data_raw_green/gt_201409.csv')
gt_1410 = pd.read_csv('data_raw_green/gt_201410.csv')
gt_1411 = pd.read_csv('data_raw_green/gt_201411.csv')
gt_1412 = pd.read_csv('data_raw_green/gt_201412.csv')

In [16]:
gt_1501 = pd.read_csv('data_raw_green/gt_201501.csv')
gt_1502 = pd.read_csv('data_raw_green/gt_201502.csv')
gt_1503 = pd.read_csv('data_raw_green/gt_201503.csv')
gt_1504 = pd.read_csv('data_raw_green/gt_201504.csv')
gt_1505 = pd.read_csv('data_raw_green/gt_201505.csv')
gt_1506 = pd.read_csv('data_raw_green/gt_201506.csv')
gt_1507 = pd.read_csv('data_raw_green/gt_201507.csv')
gt_1508 = pd.read_csv('data_raw_green/gt_201508.csv')
gt_1509 = pd.read_csv('data_raw_green/gt_201509.csv')
gt_1510 = pd.read_csv('data_raw_green/gt_201510.csv')
gt_1511 = pd.read_csv('data_raw_green/gt_201511.csv')
gt_1512 = pd.read_csv('data_raw_green/gt_201512.csv')

In [17]:
gt_1601 = pd.read_csv('data_raw_green/gt_201601.csv')
gt_1602 = pd.read_csv('data_raw_green/gt_201602.csv')
gt_1603 = pd.read_csv('data_raw_green/gt_201603.csv')
gt_1604 = pd.read_csv('data_raw_green/gt_201604.csv')
gt_1605 = pd.read_csv('data_raw_green/gt_201605.csv')
gt_1606 = pd.read_csv('data_raw_green/gt_201606.csv')
#gt_1607 = pd.read_csv('data_raw_green/gt_201607.csv')
#gt_1608 = pd.read_csv('data_raw_green/gt_201608.csv')
#gt_1609 = pd.read_csv('data_raw_green/gt_201609.csv')
#gt_1610 = pd.read_csv('data_raw_green/gt_201610.csv')
#gt_1611 = pd.read_csv('data_raw_green/gt_201611.csv')
#gt_1612 = pd.read_csv('data_raw_green/gt_201612.csv')

In [6]:
gt_1511.columns

Index([u'Unnamed: 0', u'pu_time', u'do_time', u'pu_lon', u'pu_lat', u'do_lon',
       u'do_lat', u'trip_distance', u'num_trips'],
      dtype='object')

In [7]:
gt_1503.head(2)

,Unnamed: 0,pu_time,do_time,pu_lon,pu_lat,do_lon,do_lat,trip_distance,num_trips
0,0,2015-03-01 00:02:05,2015-03-01 00:08:01,-73.961479,40.801872,-73.980721,40.775055,2.1,2.1
1,1,2015-03-01 00:02:11,2015-03-01 00:27:09,-73.979759,40.677467,-74.017097,40.710930,4.7,4.7


##### enter CB station ID ("choice") and radius ("radius").  Use Notebook 8 to process buffers in bulk.

In [37]:

choice = 3117
radius = 450

In [38]:
station_bounds = pd.read_csv('data_pre_cb/deployment_bounds.csv')
station_bounds.drop(['Unnamed: 0'],axis=1,inplace=True)
station_bounds[station_bounds.station_id == choice]

,boro_name,deployment,station_id,min_lon,min_lat,max_lon,max_lat
583,Brooklyn,8/12/2015 18:12:41,3117,-73.960356,40.73435,-73.956964,40.73693


In [39]:
bufs_3 = gp.GeoDataFrame.from_file('data_pre_cb/bufs_3.shp')
bufs_3.crs = {'init': 'epsg:2263', 'no_defs': True}
bufs_3 = bufs_3.to_crs(epsg=2263)
bufs_3.crs

bufs_c = bufs_3[(bufs_3.station_id == choice) & (bufs_3.radius == radius)]
bufs_c.reset_index(drop=True, inplace = True)
bufs_c

,boro_name,deployment,geometry,radius,station_id
0,Brooklyn,8/12/2015 18:12:41,"POLYGON ((996156.6610739313 207292.7847532183,...",450,3117


In [40]:
station_bounds[station_bounds.station_id == choice]

,boro_name,deployment,station_id,min_lon,min_lat,max_lon,max_lat
583,Brooklyn,8/12/2015 18:12:41,3117,-73.960356,40.73435,-73.956964,40.73693


In [41]:
station_bounds[station_bounds.station_id == choice].index[0]

583

##### important... enter index number ("idxx") of corresponding station:

In [42]:
idxx = 583

In [43]:
def choose_col_do(do_data):
    dop = do_data[['pu_time','do_time','do_lon','do_lat','trip_distance','num_trips']]
    #pu['num_trips'] = pu_data['Trip_distance']
    #pu.rename(columns={'lpep_pickup_datetime':'pu_time','Lpep_dropoff_datetime':'do_time',
    #                        'Pickup_longitude':'pu_lon', 'Pickup_latitude':'pu_lat','Trip_distance':'trip_distance'},
    #          inplace=True)
    return dop


#######################

def trim_bounds_do(do_data):
    
    startTime = datetime.datetime.now()

    min_lon = station_bounds.iloc[idxx][3]
    min_lat = station_bounds.iloc[idxx][4] 
    max_lon = station_bounds.iloc[idxx][5]
    max_lat = station_bounds.iloc[idxx][6]    
    
    do_trim = do_data[(do_data['do_lon'] > min_lon) & (do_data['do_lon'] < max_lon) &
                      (do_data['do_lat'] > min_lat) & (do_data['do_lat'] < max_lat)]
    
    print datetime.datetime.now() - startTime, "Shrunk Dataset", choice 
    
    startTime = datetime.datetime.now()

    geometry = [Point(xy) for xy in zip(do_trim['do_lon'], do_trim['do_lat'])]
    crs = {'init': 'epsg:4326', 'no_defs': True}
#    crs = {'init': 'epsg:2263', 'no_defs': True}
    do_data_points = gp.GeoDataFrame(do_trim, crs=crs, geometry=geometry)       
    do_data_points = do_data_points.to_crs(epsg=2263)
    
    print datetime.datetime.now() - startTime, "Created Points"

    return do_data_points

#############################

def join_do(rides_do):
    
    startTime = datetime.datetime.now()

    mg = sjoin(rides_do, bufs_c, how='inner', op='within')
    
    print datetime.datetime.now() - startTime,"Spatial join done"

    return mg
    

    
def trim_by_date_do(do_dt):

    startTime = datetime.datetime.now()
        
    do_dt['start'] = pd.to_datetime(do_dt['pu_time'])
    do_dt['end'] = pd.to_datetime(do_dt['do_time'])
    do_dt['trip_time'] = (do_dt['end'] - do_dt['start']).dt.total_seconds()/60
    do_dt.reset_index(drop=True, inplace = True)

    print datetime.datetime.now() - startTime,"Made trip time"
    
    return do_dt

    
    
def agg_do(do_ag):
    
    startTime = datetime.datetime.now()

    do_ag['do_date'] = do_ag['end'].dt.date
    data_do_ag = do_ag[['do_date','station_id','radius','num_trips','trip_distance','trip_time']].sort(['station_id','radius','do_date'])
    data_do_ag.reset_index(inplace = True)

    data_do_ag = data_do_ag.groupby(['station_id','radius','do_date']).agg({'num_trips': "count", 'trip_distance':"mean",'trip_time':"mean"})
    
    print datetime.datetime.now() - startTime,"Aggregated by date"
    
    data_do_ag.reset_index(inplace = True)

    return data_do_ag

def run_all(taxi_data):
    txd = choose_col_do(taxi_data)
    txd = trim_bounds_do(txd)
    txd = join_do(txd)
    txd = trim_by_date_do(txd)
    txd = agg_do(txd)
    
    return txd

In [25]:
gtx_do = pd.DataFrame()

In [44]:
#gtx_1301_do = run_all(gt_1301)
#gtx_1302_do = run_all(gt_1302)
#gtx_1303_do = run_all(gt_1303)
#gtx_1304_do = run_all(gt_1304)
#gtx_1305_do = run_all(gt_1305)
#gtx_1306_do = run_all(gt_1306)
#gtx_1307_do = run_all(gt_1307)

#gtx_1308_do = run_all(gt_1308)
gtx_1309_do = run_all(gt_1309)
gtx_1310_do = run_all(gt_1310)
gtx_1311_do = run_all(gt_1311)
gtx_1312_do = run_all(gt_1312)


gtx_1401_do = run_all(gt_1401)
gtx_1402_do = run_all(gt_1402)
gtx_1403_do = run_all(gt_1403)
gtx_1404_do = run_all(gt_1404)
gtx_1405_do = run_all(gt_1405)
gtx_1406_do = run_all(gt_1406)
gtx_1407_do = run_all(gt_1407)
gtx_1408_do = run_all(gt_1408)
gtx_1409_do = run_all(gt_1409)
gtx_1410_do = run_all(gt_1410)
gtx_1411_do = run_all(gt_1411)
gtx_1412_do = run_all(gt_1412)

gtx_1501_do = run_all(gt_1501)
gtx_1502_do = run_all(gt_1502)
gtx_1503_do = run_all(gt_1503)
gtx_1504_do = run_all(gt_1504)
gtx_1505_do = run_all(gt_1505)
gtx_1506_do = run_all(gt_1506)
gtx_1507_do = run_all(gt_1507)
gtx_1508_do = run_all(gt_1508)
gtx_1509_do = run_all(gt_1509)
gtx_1510_do = run_all(gt_1510)
gtx_1511_do = run_all(gt_1511)
gtx_1512_do = run_all(gt_1512)


gtx_1601_do = run_all(gt_1601)
gtx_1602_do = run_all(gt_1602)
gtx_1603_do = run_all(gt_1603)
gtx_1604_do = run_all(gt_1604)
gtx_1605_do = run_all(gt_1605)
gtx_1606_do = run_all(gt_1606)

#gtx_do = gtx_do.append(gtx_1308_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1309_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1310_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1311_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1312_do, ignore_index=True)

gtx_do = gtx_do.append(gtx_1401_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1402_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1403_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1404_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1405_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1406_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1407_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1408_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1409_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1410_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1411_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1412_do, ignore_index=True)

gtx_do = gtx_do.append(gtx_1501_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1502_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1503_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1504_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1505_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1506_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1507_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1508_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1509_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1510_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1511_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1512_do, ignore_index=True)

gtx_do = gtx_do.append(gtx_1601_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1602_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1603_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1604_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1605_do, ignore_index=True)
gtx_do = gtx_do.append(gtx_1606_do, ignore_index=True)


0:00:00.005726 Shrunk Dataset 3117
0:00:00.003381 Created Points
0:00:00.015134 Spatial join done
0:00:00.003944 Made trip time
0:00:00.009790 Aggregated by date
0:00:00.006035 Shrunk Dataset 3117
0:00:00.004794 Created Points
0:00:00.017327 Spatial join done
0:00:00.003967 Made trip time
0:00:00.009064 Aggregated by date
0:00:00.008833 Shrunk Dataset 3117
0:00:00.010863 Created Points
0:00:00.023476 Spatial join done
0:00:00.004023 Made trip time
0:00:00.009431 Aggregated by date


/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:72: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


0:00:00.011363 Shrunk Dataset 3117
0:00:00.014801 Created Points
0:00:00.027029 Spatial join done
0:00:00.003890 Made trip time
0:00:00.009045 Aggregated by date
0:00:00.018073 Shrunk Dataset 3117
0:00:00.023915 Created Points
0:00:00.037684 Spatial join done
0:00:00.003943 Made trip time
0:00:00.009149 Aggregated by date
0:00:00.015478 Shrunk Dataset 3117
0:00:00.030560 Created Points
0:00:00.047258 Spatial join done
0:00:00.006039 Made trip time
0:00:00.009998 Aggregated by date
0:00:00.024415 Shrunk Dataset 3117
0:00:00.034610 Created Points
0:00:00.051567 Spatial join done
0:00:00.004439 Made trip time
0:00:00.009245 Aggregated by date
0:00:00.028815 Shrunk Dataset 3117
0:00:00.041438 Created Points
0:00:00.052702 Spatial join done
0:00:00.004021 Made trip time
0:00:00.009562 Aggregated by date
0:00:00.023855 Shrunk Dataset 3117
0:00:00.042094 Created Points
0:00:00.059457 Spatial join done
0:00:00.004187 Made trip time
0:00:00.010505 Aggregated by date
0:00:00.021356 Shrunk Datase

In [45]:
gtx_do

,station_id,radius,do_date,num_trips,trip_distance,trip_time
0,3117,150,2013-09-20,1,1.130000,3.883333
1,3117,150,2013-09-29,1,1.020000,4.950000
2,3117,150,2013-10-06,1,0.850000,3.016667
3,3117,150,2013-10-16,1,1.840000,7.900000
4,3117,150,2013-10-18,1,0.990000,3.916667
5,3117,150,2013-10-19,1,1.340000,5.583333
6,3117,150,2013-10-23,1,1.340000,6.233333
7,3117,150,2013-10-26,1,1.530000,6.500000
8,3117,150,2013-11-01,1,1.580000,6.200000
9,3117,150,2013-11-02,1,3.790000,17.716667


In [46]:
green_3117_do = gtx_do
green_3117_do.to_csv('data_pre_green/green_3117_do.csv')